In [1]:
%cd /src/code
from typing import Dict, Any, List, Optional, Union
import subprocess
import hashlib
import os
import logging
import json
import random
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, STATUS_FAIL
from hyperopt import hp
from hyperopt.pyll import scope
from hyperopt import space_eval

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


import mlflow
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi


#TODO: set random state for sklearn
random.seed(42)
logging.basicConfig(level=logging.INFO)
pd.set_option('display.max_columns', None)
api = KaggleApi()
api.authenticate()

target_col: str = 'failure_prone'
submission_id: str = 'U8AXLZ'

/src/code


# EDA

In [2]:
X = pd.read_csv('../assets/X_train.csv', index_col='id')
y = pd.read_csv('../assets/y_train.csv', index_col='id')[target_col].astype(int)

print(X.shape)
print(X.info())
X.head()

(159090, 108)
<class 'pandas.core.frame.DataFrame'>
Index: 159090 entries, 0 to 159089
Columns: 108 entries, additions to delta_text_entropy
dtypes: float64(94), int64(14)
memory usage: 132.3 MB
None


,additions,additions_avg,additions_max,avg_play_size,avg_task_size,change_set_avg,change_set_max,code_churn_avg,code_churn_count,code_churn_max,commits_count,contributors_count,deletions,deletions_avg,deletions_max,highest_contributor_experience,hunks_median,lines_blank,lines_code,lines_comment,minor_contributors_count,num_authorized_key,num_block_error_handling,num_blocks,num_commands,num_conditions,num_decisions,num_deprecated_keywords,num_deprecated_modules,num_distinct_modules,num_external_modules,num_fact_modules,num_file_exists,num_file_mode,num_file_modules,num_filters,num_ignore_errors,num_import_playbook,num_import_role,num_import_tasks,num_include,num_include_role,num_include_tasks,num_include_vars,num_keys,num_lookups,num_loops,num_math_operations,num_names_with_vars,num_parameters,num_paths,num_plays,num_prompts,num_regex,num_roles,num_suspicious_comments,num_tasks,num_tokens,num_unique_names,num_uri,num_vars,text_entropy,delta_avg_play_size,delta_avg_task_size,delta_lines_blank,delta_lines_code,delta_lines_comment,delta_num_authorized_key,delta_num_block_error_handling,delta_num_blocks,delta_num_commands,delta_num_conditions,delta_num_decisions,delta_num_deprecated_keywords,delta_num_deprecated_modules,delta_num_distinct_modules,delta_num_external_modules,delta_num_fact_modules,delta_num_file_exists,delta_num_file_mode,delta_num_file_modules,delta_num_filters,delta_num_ignore_errors,delta_num_import_playbook,delta_num_import_role,delta_num_import_tasks,delta_num_include,delta_num_include_role,delta_num_include_tasks,delta_num_include_vars,delta_num_keys,delta_num_lookups,delta_num_loops,delta_num_math_operations,delta_num_names_with_vars,delta_num_parameters,delta_num_paths,delta_num_plays,delta_num_prompts,delta_num_regex,delta_num_roles,delta_num_suspicious_comments,delta_num_tasks,delta_num_tokens,delta_num_unique_names,delta_num_uri,delta_num_vars,delta_text_entropy
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0.0,10.0,2,27,0,0,0,0,0,0,0,0,0.0,0.0,8.0,87.0,5.0,0,0.0,0.0,0.0,5.0,9.0,5.0,1.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,0.0,5.0,0.0,1.0,17.0,5.0,0.0,0.0,0.0,0.0,0.0,9.0,293.0,11.0,0.0,1.0,6.39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0.0,6.0,1,1,0,0,0,0,0,0,0,0,0.0,0.0,24.0,103.0,59.0,0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,87.0,0.0,1.0,0.0,0.0,25.0,6.0,0.0,0.0,0.0,0.0,0.0,16.0,238.0,17.0,0.0,3.0,6.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0.0,4.0,2,12,0,0,0,0,0,0,0,0,0.0,0.0,1.0,8.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,22.0,2.0,0.0,1.0,4.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0.0,6.0,1,4,0,0,0,0,0,0,0,0,0.0,0.0,5.0,33.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,28.0,0.0,1.0,0.0,1.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,109.0,8.0,0.0,1.0,5.51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0.0,4.0,1,2,0,0,0,0,0,0,0,0,0.0,0.0,1.0,9.0,12.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,24.0,2.0,0.0,0.0,4.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [3]:
print(y.info())
y.value_counts()

<class 'pandas.core.series.Series'>
Index: 159090 entries, 0 to 159089
Series name: failure_prone
Non-Null Count   Dtype
--------------   -----
159090 non-null  int64
dtypes: int64(1)
memory usage: 2.4 MB
None


failure_prone
0    149452
1      9638
Name: count, dtype: int64

In [4]:
%%time
'''
profile = ProfileReport(
    df = pd.concat(
        [X, y],
        axis=1,
    ).sample(n=10000, random_state=1).rename(columns={'failure_prone': "Target"}),  # Reduce size
    title="Profiling Report",
    # minimal=True,  # Reduces the size by showing less graphs; Not needed in our case
)
profile.config.interactions.targets = ["Target"]  # Reduce size by disabling most interactions
profile.to_file("../assets/profiling_report.html")
'''

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.96 μs


'\nprofile = ProfileReport(\n    df = pd.concat(\n        [X, y],\n        axis=1,\n    ).sample(n=10000, random_state=1).rename(columns={\'failure_prone\': "Target"}),  # Reduce size\n    title="Profiling Report",\n    # minimal=True,  # Reduces the size by showing less graphs; Not needed in our case\n)\nprofile.config.interactions.targets = ["Target"]  # Reduce size by disabling most interactions\nprofile.to_file("../assets/profiling_report.html")\n'

In [5]:
%%time
'''
profile = ProfileReport(
    df = pd.concat(
        [X, y],
        axis=1,
    ).sample(n=30000, random_state=1).rename(columns={'failure_prone': "Target"}),
    title="Profiling Report - Full",
    explorative=True,
)

profile.to_file("../assets/profiling_report_full.html")
'''

CPU times: user 4 μs, sys: 1e+03 ns, total: 5 μs
Wall time: 7.63 μs


'\nprofile = ProfileReport(\n    df = pd.concat(\n        [X, y],\n        axis=1,\n    ).sample(n=30000, random_state=1).rename(columns={\'failure_prone\': "Target"}),\n    title="Profiling Report - Full",\n    explorative=True,\n)\n\nprofile.to_file("../assets/profiling_report_full.html")\n'

# Hyperparameter tuning

tricks for conditional spaces: https://stackoverflow.com/questions/43859465/problems-setting-up-conditional-search-space-in-hyperopt

mlflow code copied from: https://github.com/LeonardoSanBenitez/tutorial-mlflow/blob/main/code/2.0%20-%20AutoML%20with%20hyperopt.ipynb

3 runs, 8 minutes

15 runs -> +-90 minutes (nao deixei rodar até o final)

9 -> 7:04:43


current competition top is 0.940


best models up to now:
Best hyperparameters found: {'framework': 'scikit-learn~=1.4', 'learning_rate': 0.46567315723999053, 'max_depth': 22, 'model': <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>, 'n_estimators': 200, 'random_state': 0}
0.8861710200921394

{'framework': 'scikit-learn~=1.4', 'learning_rate': 0.4862713013484599, 'max_depth': 26, 'model': <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>, 'n_estimators': 550, 'random_state': 0}
0.8794579500113562



this bad preprocessing got 82%, and it took an entire night to train 3 models, wtf
{'framework': 'scikit-learn~=1.4', 'learning_rate': 0.4862713013484599, 'max_depth': 26, 'model': <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>, 'n_estimators': 550, 'preprocessing_pca': 0.892751793124233, 'preprocessing_remove_uncorrelated': 0.009103062037254122, 'preprocessing_scale': True, 'random_state': 0}



In [6]:
##################################
# Preprocessing
class PandasStandardScaler(BaseEstimator, TransformerMixin):
    '''
    Custom StandardScaler that preserves pandas DataFrame
    '''
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X: pd.DataFrame, y: pd.Series) -> 'PandasStandardScaler':
        assert isinstance(X, pd.DataFrame), "X must be a pandas DataFrame."
        if X.isnull().any().any():
            raise ValueError("X contains missing values, which are not supported.")
        self.scaler.fit(X, y)
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        assert isinstance(X, pd.DataFrame), "X must be a pandas DataFrame."
        if X.isnull().any().any():
            raise ValueError("X contains missing values, which are not supported.")
        scaled_array = self.scaler.transform(X)
        return pd.DataFrame(scaled_array, index=X.index, columns=X.columns)
    

class PandasPCA(BaseEstimator, TransformerMixin):
    '''
    Custom PCA that preserves pandas DataFrame
    '''
    n_components: float
    pca: PCA
    columns: List[str] = []

    def __init__(self, n_components: float):
        assert 0 < n_components <= 1, "n_components must be a float between 0 and 1."
        self.n_components = n_components

    def fit(self, X: pd.DataFrame, y: pd.Series) -> 'PandasPCA':
        if self.n_components is not None:
            n_components = int(X.shape[1] * self.n_components)
        else:
            n_components = None
        self.pca = PCA(n_components=n_components)
        self.pca.fit(X, y)
        self.columns = [f'PC{i+1}' for i in range(self.pca.n_components_)]
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        transformed = self.pca.transform(X)
        return pd.DataFrame(transformed, index=X.index, columns=self.columns)


class RemoveUncorrelated(BaseEstimator, TransformerMixin):
    threshold: float
    dropped_columns: list[str]
    # TODO: person is not a good correçation metric for categorical

    def __init__(self, threshold: float = 0.1):
        """
        Initialize the transformer.
        Base code generated by ChatGPT. Conversation history: https://chatgpt.com/share/673c8c6e-6f90-800f-bdaa-4767b688f923

        Args:
            threshold (float): Minimim correlation. The correlation threshold below which columns will be dropped.
        """
        assert 0 <= threshold <= 1, "Threshold must be a float between 0 and 1."
        self.threshold = threshold
        self.dropped_columns = []

    def fit(self, X: pd.DataFrame, y: pd.Series) -> 'RemoveUncorrelated':
        """
        Fit the transformer by calculating correlations between columns of X and the target y.

        Args:
            X (pd.DataFrame): The input features.
            y (pd.Series): The target variable.

        Returns:
            MyCustomTransformer: Fitted transformer.
        """
        X = X.copy()
        assert isinstance(X, pd.DataFrame), "X must be a pandas DataFrame."
        assert isinstance(y, pd.Series), "y must be a pandas Series."
        assert len(X) == len(y), "X and y must have the same length."

        # Identify constant columns with standard deviation 0
        constant_columns = X.columns[X.std() <= 0.00000001].tolist()
        if constant_columns:
            logging.debug(f"Columns with zero standard deviation: {constant_columns}")
            self.dropped_columns.extend(constant_columns)
            X = X.drop(columns=constant_columns)

        # Remove rows with null values
        clean_data = pd.concat([X, y], axis=1).dropna()
        removed_fraction = 1 - len(clean_data) / len(X)
        if removed_fraction > 0.9:
            logging.warning(f"{removed_fraction:.2%} of the dataset rows were removed due to null values ({len(clean_data)} rows remaining out of {len(X)})")
        X = clean_data.iloc[:, :-1]
        y = clean_data.iloc[:, -1]

        # Compute correlations
        correlations = X.corrwith(y, method='pearson')
        low_corr_columns = correlations[correlations.abs() < self.threshold].index.tolist()
        self.dropped_columns.extend(low_corr_columns)
        removed_fraction_cols = len(self.dropped_columns) / X.shape[1]
        if removed_fraction_cols > 0.9:
            logging.warning(f"{removed_fraction_cols:.2%} of the columns were removed due to low correlation ({X.shape[1] - len(self.dropped_columns)} columns remaining out of {X.shape[1]})")

        # Log the information
        logging.info(f"Dropping columns due to low correlation: {low_corr_columns}")
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Transform the input data by removing columns with low correlation.

        Args:
            X (pd.DataFrame): The input features.

        Returns:
            pd.DataFrame: Transformed DataFrame with low-correlation columns removed.
        """
        assert isinstance(X, pd.DataFrame), f"X must be a pandas DataFrame, got {type(X)}"
        return X.drop(columns=self.dropped_columns, errors='ignore')

    
##################################
# Training utilities
def calculate_params_hash(params: Dict[str, Any]) -> str:
    stringified = json.dumps({k:str(params[k]) for k in params}, sort_keys=True)
    return hashlib.md5(stringified.encode()).hexdigest()


def train_model(params: Dict[str, Any], X: pd.DataFrame, y: pd.Series, analyze: bool = False, analyze_n: int = 1000, analyze_features: List[int] = []) -> Any:
    '''
    @return fitted sklearn-like model (have a method `predict(X)`, etc)
    '''
    params = params.copy()
    ModelClass = params.pop('model')
    assert params.pop('framework') in ['scikit-learn~=1.4', 'xgboost~=2.1']

    pipeline_steps = []
    if 'preprocessing_remove_uncorrelated' in params:
        pipeline_steps.append(('remove_uncorrelated', RemoveUncorrelated(threshold=params.pop('preprocessing_remove_uncorrelated'))))
    if 'preprocessing_scale' in params and params['preprocessing_scale']:
        pipeline_steps.append(('standardize', PandasStandardScaler()))
        params.pop('preprocessing_scale')
    if 'preprocessing_pca' in params:
        pipeline_steps.append(('pca', PandasPCA(n_components=params.pop('preprocessing_pca'))))
    pipeline_steps.append(('model', ModelClass(**params)))
    pipeline = Pipeline(pipeline_steps)
    pipeline.fit(X, y)

    if analyze:
        try:
            shap.initjs()
            # TODO: choose the explained based on the model type
            explainer = shap.TreeExplainer(pipeline.named_steps['model'])
            explanations = explainer(X.sample(n=1000, random_state=1))
            
            # Waterfall plot of shap values for one single prediction
            # f(x): predicted target for that row
            # E[f(X)]: average target
            shap.plots.waterfall(explanations[0])
            
            # Force plot of shap values for one single prediction
            # It's like a stacked waterfall
            shap.plots.force(explanations[0])
            shap.plots.force(explanations, ordering_keys=explanations.sum(1))
            
            # Headmap (rows are instances, columns are features)
            shap.plots.heatmap(explanations, instance_order=explanations.sum(1))
            
            # Summary of feature importance
            shap.plots.bar(explanations)
            
            # Beeswarm plot (like a violin plot, points are shap values)
            shap.plots.beeswarm(explanations)
            
            # Dependence plot
            # scatter of shap (y) vs one feature
            if len(analyze_features) == 0:
                analyze_features = [explanations.abs.mean(0).values.argmax()]  # Most important feature
            for feature in analyze_features:
                shap.plots.scatter(explanations[:, feature])
    
            # TODO:
            # Return the graphs so that they can be logged by ML flow
            # Or save them: plt.savefig("shap_waterfall.png")
        except Exception as e:
            logging.error(f'Unexpected error at SHAP: {e}')
    return pipeline


def objective(params: dict):
    # The k-fold logic was generated by chatGPT: https://chatgpt.com/share/6729189d-8bec-800f-9ab3-d4ec6d9617f6
    try:
        assert isinstance(params, dict)
        assert isinstance(X, pd.DataFrame)
        assert isinstance(y, pd.Series)

        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        with mlflow.start_run(nested=True):
            mlflow.log_params(params)
            mlflow.log_param('hyperparameters_hash', calculate_params_hash(params))
            logging.info("-"*50)
            logging.info(f"Trying out params {calculate_params_hash(params)}   -> {params}")
            scores = []
            for train_ix, test_ix in kfold.split(X, y):
                train_X, test_X = pd.DataFrame(X.iloc[train_ix]), pd.DataFrame(X.iloc[test_ix])
                train_y, test_y = y.iloc[train_ix], y.iloc[test_ix]
                assert isinstance(train_X, pd.DataFrame)
                assert isinstance(test_X, pd.DataFrame)
                assert isinstance(train_y, pd.Series)
                assert isinstance(test_y, pd.Series)
                model = train_model(params, train_X, train_y)     
                y_pred = model.predict(test_X)
                score = balanced_accuracy_score(test_y, y_pred)
                scores.append(score)
                logging.info(f'Trained model on {train_X.shape[0]} rows, got score {score}')
            metric = np.mean(scores)
            logging.info(f"Final metric: {metric}")
            mlflow.log_metric("balanced_accuracy_score_mean", metric)
        return {'loss': -metric, 'status': STATUS_OK}
    except Exception as e:
        logging.exception(e)
        return {'loss': -666, 'status': STATUS_FAIL}


In [7]:
# -0.886171020092139
# d = objective({'framework': 'scikit-learn~=1.4', 'learning_rate': 0.46567315723999053, 'max_depth': 22, 'model': GradientBoostingClassifier, 'n_estimators': 400, 'random_state': 0})
# d

In [ ]:
%%time
space = hp.choice('classifier', [
    #{
    #    'model': RandomForestClassifier,
    #    'framework': 'scikit-learn~=1.4',
    #    'n_estimators': 500,#scope.int(hp.quniform('rf_n_estimators', 20, 60, 5)),
    #    'max_depth': 25,#scope.int(hp.quniform('rf_max_depth', 5, 10, 1)),
    #    'random_state':0,
    #    'preprocessing_scale': True,
    #    'preprocessing_pca': 0.8,
    #    'preprocessing_remove_uncorrelated': 0.01,#hp.uniform('rf_preprocessing_remove_uncorrelated', 0.01, 0.1),
    #},


    # Best up to now, but including preporcesing
    #{
    #    'framework': 'scikit-learn~=1.4', 
    #    'learning_rate': 0.4862713013484599, 
    #    'max_depth': 26, 
    #    'model': GradientBoostingClassifier,
    #    'n_estimators': 550,
    #    'random_state': 0,
    #    'preprocessing_scale': True,
    #    'preprocessing_pca': hp.uniform('rf_preprocessing_pca', 0.7, 1.0),   
    #    'preprocessing_remove_uncorrelated': hp.uniform('rf_preprocessing_remove_uncorrelated', 0.001, 0.015),    
    #}

    # Run 2
    #{
    #    'model': RandomForestClassifier,
    #    'framework': 'scikit-learn~=1.4',
    #    'n_estimators': scope.int(hp.quniform('rf_n_estimators', 200, 600, 50)),
    #    'max_depth': scope.int(hp.quniform('rf_max_depth', 50, 100, 10)),
    #    'random_state':0,
    #},
    #{
    #    'model': GradientBoostingClassifier,
    #    'framework': 'scikit-learn~=1.4',
    #    'learning_rate': hp.uniform('gb_learning_rate', 0.4, 1),
    #    'n_estimators': scope.int(hp.quniform('gb_n_estimators', 200, 600, 50)),
    #    'max_depth': scope.int(hp.quniform('gb_max_depth', 15, 30, 2)),
    #    'random_state': 0,
    #},

    # Run 2C
    #{
    #    'model': RandomForestClassifier,
    #    'framework': 'scikit-learn~=1.4',
    #    'n_estimators': scope.int(hp.quniform('rf_n_estimators', 50, 200, 50)),  # discrite values from 50 to 500, every 50
    #    'max_depth': scope.int(hp.quniform('rf_max_depth', 30, 100, 10)),
    #    'random_state':0,
    #    'preprocessing_scale': True,
    #    'preprocessing_pca': hp.uniform('rf_preprocessing_pca', 0.1, 0.8),
    #    'preprocessing_remove_uncorrelated': hp.uniform('rf_preprocessing_remove_uncorrelated', 0.01, 0.1),
    #},
    #{
    #    'model': GradientBoostingClassifier,
    #    'framework': 'scikit-learn~=1.4',
    #    'learning_rate': hp.uniform('gb_learning_rate', 0.01, 0.5),  # continuous range from 0.01 to 0.5
    #    'n_estimators': scope.int(hp.quniform('gb_n_estimators', 50, 200, 50)),
    #    'max_depth': scope.int(hp.quniform('gb_max_depth', 10, 30, 2)),
    #    'random_state': 0,
    #    'preprocessing_scale': True,
    #    'preprocessing_pca': hp.uniform('gb_preprocessing_pca', 0.8, 1.0),
    #    'preprocessing_remove_uncorrelated': hp.uniform('gb_preprocessing_remove_uncorrelated', 0.01, 0.1),
    #},

    ################
    # Run 1
    {
        'model': RandomForestClassifier,
        'framework': 'scikit-learn~=1.4',
        'n_estimators': scope.int(hp.quniform('rf_n_estimators', 50, 200, 50)),  # discrite values from 50 to 500, every 50
        'max_depth': scope.int(hp.quniform('rf_max_depth', 10, 100, 10)),
        'random_state': 0,
    },
    {
        'model': GradientBoostingClassifier,
        'framework': 'scikit-learn~=1.4',
        'learning_rate': hp.uniform('gb_learning_rate', 0.1, 0.5),  # continuous range from 0.01 to 0.5
        'n_estimators': scope.int(hp.quniform('gb_n_estimators', 100, 200, 50)),
        'max_depth': 10, #scope.int(hp.quniform('gb_max_depth', 3, 30, 2)),
        'random_state': 0,
    },
    {
        'model': KNeighborsClassifier,
        'framework': 'scikit-learn~=1.4',
        'n_neighbors': scope.int(hp.quniform('knn_n_neighbors', 5, 50, 2)),
    },
    {
        'model': SVC,
        'framework': 'scikit-learn~=1.4',
        'C': hp.uniform('svm_C', 0.1, 10),  # continuous range from 0.1 to 10
        'kernel': hp.choice('svm_kernel', ['poly', 'rbf']),
        #'degree': scope.int(hp.quniform('svm_degree', 2, 5, 1)),  # only used for 'poly' kernel
        #'gamma': hp.choice('svm_gamma', ['scale', 'auto']),
        'random_state': 0,
    },
    {
        'model': MLPClassifier,
        'framework': 'scikit-learn~=1.4',
        'hidden_layer_sizes': scope.int(hp.quniform('mlp_hidden_layer_sizes', 50, 200, 50)),
        'activation': hp.choice('mlp_activation', ['tanh', 'relu']),
        #'solver': hp.choice('mlp_solver', ['sgd', 'adam']),
        #'alpha': hp.uniform('mlp_alpha', 0.0001, 0.01),
        #'learning_rate': hp.choice('mlp_learning_rate', ['invscaling', 'adaptive']),
        'random_state': 0,
    },
    {
        'model': XGBClassifier,
        'framework': 'xgboost~=2.1',
        'learning_rate': hp.uniform('xgb_learning_rate', 0.3, 0.5),
        'n_estimators': scope.int(hp.quniform('xgb_n_estimators', 100, 200, 50)),
        'max_depth': 10, #scope.int(hp.quniform('xgb_max_depth', 10, 30, 2)),
        'random_state': 0,
    }
])

mlflow.set_tracking_uri('http://mlflow:5000')
mlflow.set_experiment("round_1")
with mlflow.start_run(description='Parent run for hyperopt', nested=True):
    best_hyperparameters: Dict[str, Any] = space_eval(space, fmin(
        fn=objective,             # Objective function to minimize
        space=space,              # Hyperparameter space
        algo=tpe.suggest,         # Optimization algorithm (Tree of Parzen Estimators)
        max_evals=50,             # Number of evaluations
        trials=Trials()           # Store trial history for later inspection
    ))

    assert type(best_hyperparameters) == dict
    print("Best hyperparameters found:", best_hyperparameters)

2024/12/02 19:20:58 INFO mlflow.tracking.fluent: Experiment with name 'round_1' does not exist. Creating a new experiment.


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.006179 seconds
INFO:hyperopt.tpe:TPE using 0 trials
INFO:root:--------------------------------------------------
INFO:root:Trying out params f59fca057362e184f1481c0bee9aa626   -> {'framework': 'scikit-learn~=1.4', 'model': <class 'sklearn.neighbors._classification.KNeighborsClassifier'>, 'n_neighbors': 14}
INFO:root:Trained model on 106060 rows, got score 0.6668364753056043
INFO:root:Trained model on 106060 rows, got score 0.671391419579789
INFO:root:Trained model on 106060 rows, got score 0.6780228615256583
INFO:root:Final metric: 0.6720835854703505
2024/12/02 19:21:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run unique-sloth-325 at: http://mlflow:5000/#/experiments/450078657812037678/runs/540b11f6ef864cf4b0c0eac57a37bf46.

2024/12/02 19:21:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/450078657812037678.



  2%|▏         | 1/50 [00:44<36:38, 44.87s/trial, best loss: -0.6720835854703505]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.004795 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss -0.672084
INFO:root:--------------------------------------------------
INFO:root:Trying out params 27109dc363ec0797699e2cc3328b3834   -> {'C': 5.880647620819095, 'framework': 'scikit-learn~=1.4', 'kernel': 'poly', 'model': <class 'sklearn.svm._classes.SVC'>, 'random_state': 0}


# Train final model
expects a dictionary `best_hyperparameters`

In [ ]:
%%time
# TODO: register at mlflow
best_hash = calculate_params_hash(best_hyperparameters)
print(f'Search experiment by param hash in the UI: http://localhost:5001/#/experiments/127862172636814249?searchFilter=params.hyperparameters_hash+%3D"{best_hash}"')

model = train_model(best_hyperparameters, X, y, analyze=True)
model

In [ ]:
importances = model.named_steps['model'].feature_importances_
columns = model.named_steps['pca'].columns

# Convert importances to percentage
importances_percentage = importances / importances.sum() * 100

# Create a dataframe for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': columns,
    'Importance (%)': importances_percentage
}).sort_values(by='Importance (%)', ascending=False)

# Display the dataframe
feature_importances_df.head(10)  # Display top 10 features

In [ ]:
# TODO: better eval this model (confusion matrix, etc)

# Submit best model
Expects a variable `model`, with a method `predict` that follows a sklean-like interface, and a dictionary `hyperparameters`

In [ ]:
assert model is not None
assert hasattr(model, 'predict')

X_test = pd.read_csv('../assets/X_test.csv', index_col='id')
assert X_test.shape[1] == X.shape[1]
assert all(X_test.columns == X.columns)

y_test_hat = model.predict(X_test)
assert len(y_test_hat.shape) == 1
assert y_test_hat.shape[0] == X_test.shape[0]

assert y_test_hat.min() == 0
assert y_test_hat.max() == 1
assert y_test_hat.dtype == np.dtype('int64')

In [ ]:
# Submission params
competition = 'ppcu-data-mining-and-machine-learning-2024'
file_name = f'../assets/y_test_submission_{submission_id}.csv'
message = f"test trained submission"
message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
message += f"; hyperparameters_hash: {calculate_params_hash(best_hyperparameters)}"

# Run inference
if os.path.exists(file_name):
    os.remove(file_name)
pd.DataFrame(y_test_hat, index=X_test.index, columns=["failure_prone"]).to_csv(file_name, index=True)
assert os.path.exists(file_name)

# Submit the file to the competition
#api.competition_submit(file_name, message, competition)
